# Setup

In [ ]:
# setup to see the execution time in each cell

!pip install ipython-autotime
!pip install directory_structure
# !pip install wandb
%load_ext autotime

In [ ]:
import pandas as pd
import os
import glob
import PIL
from PIL import Image
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt

In [ ]:
DRIVER_ROOT_DIR = "/kaggle/input/" # HC Directory
# DRIVER_ROOT_DIR = "/content/drive/MyDrive/DTSC 870/Code" #MT Directory

# no augmentation
# DATASET_02_TRAIN = DRIVER_ROOT_DIR + "fer2013/02_FER/train"
# DATASET_02_TEST = DRIVER_ROOT_DIR + "fer2013/02_FER/test"

# augmentation v.1
# DATASET_02_TRAIN = DRIVER_ROOT_DIR + "fer2013-aug/Aug_train"
# DATASET_02_TEST = DRIVER_ROOT_DIR + "fer2013-aug-test/Aug_test"

# augmentation v.2
DATASET_02_TRAIN = DRIVER_ROOT_DIR + "fer2013-aug-train-2/Aug_train_2"
# DATASET_02_TEST = DRIVER_ROOT_DIR + "fer2013-aug-test-2/Aug_test_2"
DATASET_02_TEST = DRIVER_ROOT_DIR + "d/datasets/huihenrychen/fer2013-aug-test-2/Aug_test_2"

## Utility Functions

In [ ]:
def get_data_df(dir):
    # modified code from: https://www.kaggle.com/namgalielei/simple-load-images-and-count-number-of-each-class

    train_df = pd.DataFrame()

    trainset = glob.glob(dir)

    train_df['file'] = [img.split("/")[-1] for img in trainset]
    train_df['class'] = [img.split("/")[-2] for img in trainset]

    return train_df

In [ ]:
def generate_set(df, dir, classes_):

    # new_df = pd.DataFrame()
    pixels = []
    class_ = []

    # trainset = glob.glob(dir)
    for i in range(len(df.index)):
        # get the absolute img path
        # e.g., Brain_tumor_images/<train or test>/<class label>/<file name>
        path = dir + "/" + df.iloc[i]["class"] + "/" +df.iloc[i]["file"]
        # print(img)
        img = Image.open(path)
        # print("Img: {} \tClass: {}".format(np.array(img).flatten(), df.iloc[i]["class"]))
        pixels.append(cp.asnumpy(cp.array(img)).flatten())
        # pixels.append(np.array(img))

        # y_true encoding here
        class_.append(classes_.index(df.iloc[i]["class"]))

        # end loop here

    # return train_df
    return pixels, class_

## Generate the train and test sets

In [ ]:
fer_df_train = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
fer_df_test = get_data_df(DATASET_02_TEST+"/*/*.jpg")

In [ ]:
classes = fer_df_train["class"].unique().tolist()
classes
# classes.index("surprise")

In [ ]:
x_train, y_train = generate_set(fer_df_train, DATASET_02_TRAIN, classes)
x_test, y_test = generate_set(fer_df_test, DATASET_02_TEST, classes)

### EDA

In [ ]:
# fer_df_test.sample(10)

In [ ]:
# fer_df_train.shape

In [ ]:
x_train_df = pd.DataFrame()
x_train_df['class'] = y_train

# x_train_df

In [ ]:
x_train_df["class"].hist()

In [ ]:
# print(np.array(x_train).shape)
# print(np.array(x_test).shape)

# Class weights

In [ ]:
print("Classes: {}".format(classes))
print("X train shape: {}".format(np.array(x_train).shape))
print("X test shape: {}".format(np.array(x_test).shape))

# total samples
N = np.array(x_train).shape[0] + np.array(x_test).shape[0]
print("Total sample size: {}".format(N))


# total class sample count
train_set_count = np.bincount(np.array(y_train))
test_set_count = np.bincount(np.array(y_test))
print("Train set: {}".format(train_set_count))
print("Test set: {}".format(test_set_count))
# print(type(test_set_count))


# assign weights to each classes
c_weight = {}
class_len = len(classes)
for i in range(len(classes)):
    c_weight[i] = N/(class_len*(train_set_count[i] + test_set_count[i]))
    
print("The class weights: {}".format(c_weight))

# Feature Engineering

In [ ]:
# from sklearn.decomposition import PCA
from cuml.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import cv2
from skimage.feature import local_binary_pattern
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from matplotlib.pyplot import figure

## PCA

In [ ]:
pca = PCA()
x_train_pca = pca.fit_transform(np.array(x_train, dtype=np.float32))
x_test_pca = pca.transform(np.array(x_test, dtype=np.float32))

In [ ]:
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
exp_var_cumul_round = np.round_(exp_var_cumul, decimals = 4)

num_comp = range(1, exp_var_cumul_round.shape[0] + 1)

print(exp_var_cumul_round)

### PCA Visualization

In [ ]:
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    x_train_pca,
    # x_train_df,
    labels=labels,
    # dimensions=(0, 1),
    dimensions=range(4),
    # color=x_train_df['class'],
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
# get a list of # of components
num_comp = range(1, exp_var_cumul_round.shape[0] + 1)

per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)

fig = px.line(
    x=num_comp,
    y=exp_var_cumul_round,
    labels={"x": "# Components", "y": "Cumulative Explained Variance"},
    title = "# of components V.S. variance",
    # markers=True
)

fig.show()

In [ ]:
# selected_exp_var = [0.7, 0.8, 0.9, 0.95, 0.97, 0.99, 1.0]
# selected_num_comp_aug = [14, 32, 108, 269, 446, 940, 2265]

In [ ]:
per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)
# labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]

fig = px.bar(
    x=range(1,len(per_var)+1), 
    y=per_var, 
    title='Scree Plot of # of components and % of explained variance',
    labels={"x": "# Components", "y": "% of Explained Variance"},
)
fig.show()

# Model

## Setup

In [ ]:
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from numpy import mean

from cuml.svm import SVC
# from sklearn.svm import SVC

In [ ]:
def generate_PCA_set(comp, x_train, x_test):
    pca = PCA(n_components=comp)
    __x_train__ = pca.fit_transform(x_train)
    __x_test__ = pca.transform(x_test)
    return __x_train__, __x_test__

In [ ]:
def get_model():
    return SVC(random_state=1, multiclass_strategy="ovr")

def get_model2():
    return SVC(random_state=1, multiclass_strategy="ovr", class_weight=c_weight)

In [ ]:
def feature_scale(_x_train_, _x_test_):
    sc = StandardScaler()
    
    train_sc = sc.fit_transform(_x_train_)
    test_sc = sc.transform(_x_test_)
    
    return train_sc, test_sc


def feature_scale2(_x_train_):
    sc = StandardScaler()
    
    train_sc2 = sc.fit_transform(_x_train_)
    
    return train_sc2

## Train

In [ ]:
selected_exp_var = [0.7, 0.8, 0.9, 0.95, 0.97, 0.99, 1.0]

# none img augmentation
selected_num_comp = [13, 32, 104, 256, 425, 904, 2304]

# img augmentation
selected_num_comp_aug = [14, 32, 108, 269, 446, 940, 2265]

### Training with No Class weights

In [ ]:
svm_pca_experiment = list()

for i in range(len(selected_exp_var)):
    
#     temp_train, temp_test = generate_PCA_set(selected_num_comp[i], x_train, x_test)
    temp_train, temp_test = generate_PCA_set(selected_num_comp_aug[i], np.array(x_train, dtype=np.float32), np.array(x_test, dtype=np.float32))
    
    svm_pca_experiment.append({
        "num_compon": selected_num_comp[i],
        "variance": selected_exp_var[i],
        "model": get_model(),
        "x_train": temp_train,
        "x_test": temp_test
    })

### Training with Class weights

In [ ]:
# svm_pca_experiment2 = svm_pca_experiment

# for i in range(len(selected_exp_var)):
    
#     svm_pca_experiment2[i]["model"] = get_model2()

In [ ]:
def svm_pca_training(arr_, y_train, y_test):
    uw_acc_test_result = list()
    w_acc_test_result = list()
    y_pred = list()
    
    for i in range(len(arr_)):
        # scaling
        x_train_sc, x_test_sc = feature_scale(arr_[i]["x_train"], arr_[i]["x_test"])

        x_train_sc, x_test_sc = np.array(x_train_sc), np.array(x_test_sc)
        y_train = np.array(y_train, dtype=np.float32)

        # train the model
        # model = SVC(C=0.1, gamma=0.01, kernel='rbf', random_state=1, multiclass_strategy="ovr")
        model = arr_[i]["model"]
        
        # model = SVC(decision_function_shape="ovr", kernel="rbf", random_state=13)
        model.fit(x_train_sc, y_train)

        # predict the train set
        y_trainHat = model.predict(x_train_sc)

        # predict the train set
        y_testHat = model.predict(x_test_sc)
        
        y_pred.append(y_testHat)

        # compute the unweighted test accuracy
        uw_acc = balanced_accuracy_score(y_test, y_testHat)
        uw_acc_test_result.append(uw_acc)
        print("The unweighted accuracy: {}".format(uw_acc))

        # compute the weighted test accuracy
        w_acc = accuracy_score(y_test, y_testHat)
        w_acc_test_result.append(w_acc)
        print("The weighted accuracy: {}".format(w_acc))
        print("\n")
        
    
    return uw_acc_test_result, w_acc_test_result, y_pred



def svm_pca_training2(arr_, y_train, y_test):
    uw_acc_test_result = list()
    w_acc_test_result = list()
    y_pred = list()

    # scaling
    x_train_sc, x_test_sc = feature_scale(arr_["x_train"], arr_["x_test"])

    x_train_sc, x_test_sc = np.array(x_train_sc), np.array(x_test_sc)
    y_train = np.array(y_train, dtype=np.float32)

    # train the model
    # model = SVC(C=0.1, gamma=0.01, kernel='rbf', random_state=1, multiclass_strategy="ovr")
    model = arr_["model"]

    # model = SVC(decision_function_shape="ovr", kernel="rbf", random_state=13)
    model.fit(x_train_sc, y_train)

    # predict the train set
    y_trainHat = model.predict(x_train_sc)

    # predict the train set
    y_testHat = model.predict(x_test_sc)

    y_pred.append(y_testHat)

    # compute the unweighted test accuracy
    uw_acc = balanced_accuracy_score(y_test, y_testHat)
    uw_acc_test_result.append(uw_acc)
    print("The unweighted accuracy: {}".format(uw_acc))

    # compute the weighted test accuracy
    w_acc = accuracy_score(y_test, y_testHat)
    w_acc_test_result.append(w_acc)
    print("The weighted accuracy: {}".format(w_acc))
    print("\n")
        
    
    return uw_acc_test_result, w_acc_test_result, y_pred

In [ ]:
# uw_acc_result, w_acc_result, y_pred = svm_pca_training2(svm_pca_experiment[2], y_train, y_test)
uw_acc_result, w_acc_result, y_pred = svm_pca_training(svm_pca_experiment, y_train, y_test)

# uw_acc_result, w_acc_result, y_pred = svm_pca_training(svm_pca_experiment2, y_train, y_test)

## Model Train

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
np.unique(c_weight)

In [ ]:
def train_model2(arr_, y_train, y_test):
    y_pred2 = list()
    
    # scaling
    x_train_sc, x_test_sc = feature_scale(arr_["x_train"], arr_["x_test"])

    x_train_sc, x_test_sc = np.array(x_train_sc), np.array(x_test_sc)
    y_train = np.array(y_train, dtype=np.float32)

    # train the model
    # model = SVC(C=0.1, gamma=0.01, kernel='rbf', random_state=1, multiclass_strategy="ovr")
    model = OneVsRestClassifier(SVC(random_state=1, multiclass_strategy="ovr"))

    # model = SVC(decision_function_shape="ovr", kernel="rbf", random_state=13)
    model.fit(x_train_sc, y_train)

    # predict the train set
    y_trainHat = model.predict(x_train_sc)

    # predict the train set
    y_testHat = model.predict(x_test_sc)

    y_pred2.append(y_testHat)
    
    # calculate the gamma value
    n_features = x_train_sc.shape[1]
    gamma = 1/(n_features * x_train_sc.var())
    print('Gamma: %.09f' % gamma)

    # compute the unweighted test accuracy
    uw_acc = balanced_accuracy_score(y_test, y_testHat)
    # uw_acc_test_result.append(uw_acc)
    print("The unweighted accuracy: {}".format(uw_acc))

    # compute the weighted test accuracy
    w_acc = accuracy_score(y_test, y_testHat)
    # w_acc_test_result.append(w_acc)
    print("The weighted accuracy: {}".format(w_acc))
    print("\n")
    
    return model, y_pred2
    

model2, y_pred2 = train_model2(svm_pca_experiment[2], y_train, y_test)

# Result Analysis

In [ ]:
# no augmentation
uw_acc_result = [0.34913545500412424, 0.410326622837653, 0.4606871805554088, 0.4416885384206618, 0.42643337815184973, 0.39500145483379384, 0.35858548421567005]
w_acc_result = [0.36458623572025634, 0.44831429367511844, 0.49080523822791866, 0.48021733073279466, 0.4653106714962385, 0.4395374756199498, 0.40025076623014766]

# augmentation 1
# uw_acc_result = [0.30401166995891815, 0.36459252004568043, 0.3921733369630049, 0.37487205858731293, 0.3576369966351716, 0.327007717935318, 0.2870783470228984]
# w_acc_result = [0.34159933129005293, 0.40707718027305656, 0.43076065756478127, 0.42073000835887436, 0.40373363053775424, 0.3757314015045974, 0.3354694901086654]

# augmentation 2
# uw_acc_result = [0.401166992591892, 0.36459252004568043, 0.40217333978230513, 0.38487135815731293, 0.33552365266351716, 0.303055725934518, 0.2752783475228984]
# w_acc_result = [0.4159253129002493, 0.40707718027305656, 0.44076985756473527, 0.40073550832551436, 0.38075363053775424, 0.349514015045974, 0.3414414901063654]

In [ ]:
# import plotly.graph_objects as go

# fig = go.Figure()
# fig.add_trace(go.Line(
#     x=selected_exp_var,
#     y=uw_acc_result,
#     mode='lines',
#     name='Unweighted Test Accuracy',
#     marker=dict(
#         color='red',
#         size=10
#     ))
# )

# fig.add_trace(go.Line(
#     x=selected_exp_var,
#     y=w_acc_result,
#     mode='lines',
#     name='Weighted Test Accuracy',
#     marker=dict(
#         color='green',
#         size=10
#     ))
# )


# fig.update_layout(
#     title="SVM accuracy of different PCA variance (no aug)",
#     xaxis_title="SVM % of Explained Variance",
#     yaxis_title="Accuracy (%)"
# )
# fig.update_traces(mode='markers+lines')

# fig.show()

In [ ]:
# plt.bar(x="unweighted", height=uw_acc)
# plt.bar(x="weighted", height=w_acc)
# plt.title("HoG on img augmentation SVM accuracy")
# plt.ylabel("Accuracy (%)")
# plt.xlabel("SVM accuracy type")
# plt.show()

In [ ]:
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from cuml.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

import plotly.express as px

In [ ]:
# _y_pred_  = list()
# for item in y_pred:
# #     print(item)
#     _y_pred_.append(item)
    
# y_pred_ = np.array(_y_pred_, dtype=np.int32)
# y_pred_[0]

In [ ]:
np.array(y_pred2)[0]

In [ ]:
for i in range(len(classes)):
    print("[{}: {}]".format(i, classes[i]), end=", ")
    

# confM = list()
# for i in range(len(svm_pca_experiment)):
index = 2
i = svm_pca_experiment[index]

y_pred_ = np.array(y_pred2, dtype=np.int32)
y_test_ = np.array(y_test, dtype=np.int32)

conf_matrix = cp.asnumpy(confusion_matrix(y_test_, y_pred_[0], normalize="true"))
conf_matrix= np.around(conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis], decimals=2)
# conf_matrix = plot_confusion_matrix(model2, y_test_, y_pred_, normalize="true", display_labels=classes)
# conf_matrix = multilabel_confusion_matrix(np.array(y_test_), np.array(y_pred2)[0], labels=classes)
confM = conf_matrix


# confM.append(conf_matrix)

t = "Normalized confusion matrix for SVM with " + str(svm_pca_experiment[index]["num_compon"]) + " PCA comp with " + str(svm_pca_experiment[index]["variance"]) + " variance"
fig = px.imshow(conf_matrix, text_auto=True, title=t, x=classes, y=classes)
# fig.update_xaxes(dtick=classes)
fig.show()

# plt.title(t)
# plt.show()
# disp.confusion_matrix

In [ ]:
# print(conf_matrix)
# row1_sum = np.array(disp.confusion_matrix[0]).sum()
# row2_sum = np.array(disp.confusion_matrix[1]).sum()

# acc_Normal = (disp.confusion_matrix[0][0] / row1_sum)*100
# acc_Tumor = (disp.confusion_matrix[1][1] / row2_sum)*100

per_class_acc = list()

for i in range(len(classes)):
    row_sum = np.array(confM[i]).sum()
    acc = (conf_matrix[i][i]/row_sum)*100
    per_class_acc.append(acc)
    
for i in range(len(classes)):
    print("[{}: {} - {}]".format(i, classes[i], per_class_acc[i]), end=", ")
    

In [ ]:
from sklearn.metrics import roc_curve, auc
from matplotlib.pyplot import figure

In [ ]:
model2.get_params()

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
type(y_test_)

In [ ]:
temp = list()
for item in y_pred2[0]:
    temp.append(int(item))

In [ ]:
y_pred2 = np.array(y_pred2).astype(int)

In [ ]:
lb = LabelBinarizer()
# temp = np.array(y_pred2[0])
enc_y_pred2 = lb.fit_transform(y_pred2[0])
enc_y_test_ = lb.fit_transform(y_test_)

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(classes)):
    fpr[i], tpr[i], _ = roc_curve(enc_y_test_[:,i], enc_y_pred2[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(enc_y_test_.ravel(), enc_y_pred2.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
# Plot ROC curve
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]))
for i in range(len(classes)):
    plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                   ''.format(i, roc_auc[i]))

# figure(figsize=(8, 6), dpi=80)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(bbox_to_anchor=(1.04,1), borderaxespad=0)
plt.show()

In [ ]:
# classes

# for i in range(len(svm_pca_experiment)):
print("Classification report for SVM with " + str(svm_pca_experiment[i]["num_compon"]) + " PCA comp with " + str(svm_pca_experiment[i]["variance"]) + " variance")
y_pred_ = y_pred2[0]
print(classification_report(y_test, y_pred_, target_names=classes))